In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
import pymysql

In [2]:
# URL of page to be scraped
url = 'http://sedimentality.com/drinking-wine/list-of-wine-and-food-pairings/'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   List of wine and food pairings | Sedimentality
  </title>
  <!-- All in One SEO Pack 2.10.1 by Michael Torbert of Semper Fi Web Design[252,305] -->
  <meta content="&quot;What wine goes with Captain Crunch?&quot; -George Karlin Pairing wines and food is too often stressed about (and debated). True, in these days we no longer abide by rules like &quot;ONLY white with fish&quot; (ever had a coriander and coffee encrusted ahi with Syrah? Amazing!) but we are often confused about how to pair certain flavors and elements of a dish. The following are some guidelines for pairing a few common wines. Remember, pairing is a funny thing, because every dish will have more than just one component. You might try to pair a wine with chicken… but it's not JUST go

In [6]:
varietals = soup.find_all('h2')

In [7]:
varietals_ls = []

for varietal in varietals:
    varietals_ls.append(varietal.text)

varietal_sr = pd.Series(varietals_ls)
varietal_sr

0        Sauvignon Blanc
1             Chardonnay
2               Riesling
3             Pinot Noir
4                  Syrah
5                 Merlot
6    Cabernet Sauvignon 
dtype: object

In [8]:
pairs = soup.find_all('strong')

keys_ls = ["Varietals"]

for pair in pairs:
    word = pair.text
    if word.endswith(":"):
        word = word.replace(":","")
        if word not in keys_ls:
            keys_ls.append(word)

keys_ls.pop()
            
df = pd.DataFrame(columns=keys_ls)
df["Varietals"] = varietal_sr
df

,Varietals,Cheese/nuts,Meat/poultry,Seafood,Fruits and Veggies,Herbs and Spices,Sauces,Desserts
0,Sauvignon Blanc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chardonnay,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Riesling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pinot Noir,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Syrah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Merlot,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cabernet Sauvignon,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
notes = soup.find_all('p')

notes_ls = []

for note in notes:
    notes_ls.append(str(note))

foods_ls = []

for note in notes_ls:
    if "</strong>" in note:
        note = note.replace(":","")
        note = note[(note.find("</strong>")+len("</strong>")):note.find("</p>")]
        note = note.strip()
        foods_ls.append(note)

keys_len = len(keys_ls) - 1
        
for i in range(0,keys_len):
    for j in range(0,len(varietals_ls)):
        df.loc[j,keys_ls[i+1]] = foods_ls[i+(j*keys_len)]

df

,Varietals,Cheese/nuts,Meat/poultry,Seafood,Fruits and Veggies,Herbs and Spices,Sauces,Desserts
0,Sauvignon Blanc,"feta, goat cheese, pine nuts","chicken, turkey, pork","fatty white fish, oysters, scallops, lobster, ...","citrus, green apple, asparagus","chives, tarragon, cilantro",citrus and light cream sauces,"sorbet, key lime pie, meringue, mango"
1,Chardonnay,"mild, semi-soft cheeses with unoaked Chardonna...","veal, chicken, pork","halibut, shrimp, crab, lobster","potato, apple, squash, mango","tarragon, sesame, basil","cream sauces, pesto","banana bread, vanilla pudding"
2,Riesling,"Havarti, gouda, candied walnuts or pecans","smoked sausage, duck, foie gras","sea bass, trout","apricots, chili peppers, pears","rosemary, ginger, Thai or Indian spices","BBQ, spicy, chutney","apple pie, caramel sauce"
3,Pinot Noir,"goat cheese, brie, walnuts","lamb, sausage, filet mignon, chicken","ahi tuna, salmon","mushrooms, dried fruits, figs, strawberries","truffle, nutmeg, cinnamon, clove","mushroom sauces, light-medium red sauces","creme brulee, white chocolate"
4,Syrah,"sharp cheddar, Roquefort/bleu-veined cheeses; ...","roasted game, pepperoni, spicy sausage, braise...","ahi tuna, salmon","currants, stewed tomatoes, beets","oregano, sage","BBQ, heavy red sauces","black forest cake, rhubarb pie, coffee-based d..."
5,Merlot,"Parmesan, Pecorino-Romano, chestnuts, walnuts","grilled meats, steak","grilled meatier fish, ahi tuna","caramelized onions, tomatoes, plums","mint, rosemary, juniper","bolognese, bearnaise","dark chocolate, berries, fondue"
6,Cabernet Sauvignon,"cheddar, gorgonzola, walnuts","venison, rib eye, beef stew",grilled ahi tuna,"black cherries, tomatoes, broccoli","rosemary, juniper, lavender","brown sauce, tomato sauce",bittersweet chocolate


In [10]:
df.to_csv("wine_pairings.csv")

In [11]:
rds_connection_string = "root:j3RKdha!@localhost:3306/etl_project"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [12]:
df.to_sql(name='wine_pairing', con=engine, if_exists='append', index=False)

In [13]:
# Kim's Transformed Wine Data
csv_file = "wine_data.csv"
wine_data_df = pd.read_csv(csv_file)

wine_data_df = wine_data_df.rename(columns={"Unnamed: 0": "id"})
wine_data_df = wine_data_df.set_index("id")
wine_data_df = wine_data_df[["country", "designation", "points", "price", "province", "region_1", "variety", "winery"]]

wine_data_df.head()

,country,designation,points,price,province,region_1,variety,winery
id,,,,,,,,
0,US,Martha's Vineyard,96,235.0,California,Napa Valley,Cabernet Sauvignon,Heitz
1,Spain,Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sauvignon Blanc,Macauley
3,US,Reserve,96,65.0,Oregon,Willamette Valley,Pinot Noir,Ponzi
4,France,La Brûlade,95,66.0,Provence,Bandol,Provence red blend,Domaine de la Bégude


In [14]:
wine_data_df.to_sql(name='wine_data', con=engine, if_exists='append', index=False)

'Cabernet Sauvignon '